In [1]:
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import os

In [3]:
train_df = pd.read_csv(os.path.join('data', "train.csv"))
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

train_df = train_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin']).dropna()
train_df["Sex"] = LabelEncoder().fit_transform(train_df["Sex"])
train_df["Embarked"] = LabelEncoder().fit_transform(train_df["Embarked"])
train_df['Age'] = StandardScaler().fit_transform(train_df['Age'].values.reshape(-1, 1))
train_df['bias'] = np.ones(train_df.shape[0])

In [5]:
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,bias
0,0,3,1,-0.527669,1,0,2,1.0
1,1,1,0,0.577094,1,0,0,1.0
2,1,3,0,-0.251478,0,0,2,1.0
3,1,1,0,0.369951,1,0,2,1.0
4,0,3,1,0.369951,0,0,2,1.0
...,...,...,...,...,...,...,...,...
885,0,3,0,0.646142,0,5,1,1.0
886,0,2,1,-0.182430,0,0,2,1.0
887,1,1,0,-0.734812,0,0,2,1.0
889,1,1,1,-0.251478,0,0,0,1.0


In [6]:
from sklearn.model_selection import train_test_split

X = train_df.drop(columns=['Survived'])
y = train_df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.15, random_state=69)

In [7]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((605, 7), (107, 7), (605, 1), (107, 1))

In [8]:


def MSELoss(y_true: np.ndarray, y_pred: np.ndarray) -> np.float32:
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return (1/len(y_true)) * np.sum((y_pred - y_true) ** 2)

def MAELoss(y_true: np.ndarray, y_pred: np.ndarray) -> np.float32:
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return (1/len(y_true)) * np.sum(np.abs((y_pred - y_true)))

def accuracy_with_thresholding(y_true: np.ndarray, y_pred: np.ndarray, threshold = 0.5):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return np.sum(
        y_true == (y_pred > threshold).astype(np.int32)
    ) / len(y_true)

def sigmoid(z):
    return 1/(1 + np.exp(-z))


In [9]:
import torch
import numpy as np

In [10]:
X = np.array([1])
Y = np.array([0])
Z = X + Y
X ,Y , Z

(array([1]), array([0]), array([1]))

In [11]:
x = torch.tensor([1.0], requires_grad=True)
y = torch.tensor([0.0], requires_grad=True)
z = (x + y) **2
x, y, z

(tensor([1.], requires_grad=True),
 tensor([0.], requires_grad=True),
 tensor([1.], grad_fn=<PowBackward0>))

In [12]:
z.backward()

In [13]:
x.grad

tensor([2.])

In [14]:
y.grad

tensor([2.])

In [15]:
# Dataset

# Dataloader

# Model

# Traing-Valid Loop

# Test


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class TitanicDataset(Dataset):
    def __init__(self, df: pd.DataFrame, target: str, transforms=None):
        super().__init__()
        self.df = df
        self.features = df.drop(columns=[target]).values
        self.target = df[target].values.reshape(-1, 1)
        self.transforms = transforms

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        X = torch.from_numpy(self.features[idx,:]).float()
        y = torch.from_numpy(self.target[idx,:]).float()

        if self.transforms:
            X = self.transforms(X)

        return (X, y)


In [19]:
training_df = pd.DataFrame(X_train, columns=train_df.columns.remove("Survived"))
training_df['Survived'] = y_train
training_df

train_dataset = TitanicDataset(training_df, target='Survived')

AttributeError: 'Index' object has no attribute 'remove'

In [ ]:
val_df = pd.DataFrame(X_test, columns=train_df.columns - ["Survived"])
val_df['Survived'] = y_test
val_df

val_dataset = TitanicDataset(val_df, target='Survived')

In [ ]:
len(train_dataset), len(val_dataset)

NameError: name 'train_dataset' is not defined

In [ ]:
train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

NameError: name 'train_dataset' is not defined

In [20]:
class TitanicModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(7, 3)
        self.linear2 = torch.nn.Linear(3, 1)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        return torch.sigmoid(self.linear2(x))

In [21]:
# Training Loop
# Loss, Optimizer, epochs, learning rate, seed

torch.manual_seed(420)
model = TitanicModel()
learning_rate = 0.001
epochs = 1000
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
# model.to('cuda')
max_accuracy = 0
for i in range(epochs):
    for idx, (X_batch, y_batch) in enumerate(train_data_loader):
        # X_batch, y_batch = X_batch.to('cuda'), y_batch.to('cuda')
        model.train()

        optimizer.zero_grad()
        y_pred = model(X_batch)
        # y_pred = y_pred.to('cpu')
        # y_batch = y_batch.to('cpu')
        loss_value = loss(y_pred, y_batch)
        loss_value.backward()
        optimizer.step()

    # Validate
    if i % 100 == 0:
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_accuracy = 0
            for X_val, y_val in val_data_loader:
                y_pred = model(X_val)
                val_loss += loss(y_pred, y_val)
                val_accuracy += accuracy_with_thresholding(y_val.numpy(), y_pred.numpy())

            val_accuracy /= len(val_data_loader)
            val_loss /= len(val_data_loader)

            if val_accuracy > max_accuracy:
                max_accuracy = val_accuracy

            print(f"Epoch {i} - Training Loss: {loss_value.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy}")

NameError: name 'train_data_loader' is not defined

In [ ]:
max_accuracy

# Mnist

In [23]:
print('Penis')


Penis


In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [27]:
transform = ToTensor()
mnist_ds = MNIST(root = 'data' ,download=True, transform=transform)
train_ds, val_ds = torch.utils.data.random_split(mnist_ds, [0.85, 0.15])

image, label = train_ds[0]


In [28]:
image

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [29]:
train_data_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_data_loader = DataLoader(val_ds, batch_size=256, shuffle=False)

# Mnist

In [30]:
#Model
class MnistModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(28*28, 512)
        self.linear2 = torch.nn.Linear(512, 128)
        self.linear3 = torch.nn.Linear(128, 32)
        self.linear4 = torch.nn.Linear(32, 10)

    def forward(self, a):
        a = torch.relu(self.linear1(a))
        a = torch.relu(self.linear2(a))
        a = torch.relu(self.linear3(a))
        a = torch.sigmoid(self.linear4(a))
        return torch.softmax(a, 1)

# Training Loop
# Loss, Optimizer, epochs, learning rate, seed

torch.manual_seed(420)
model = MnistModel()
learning_rate = 0.01
epochs = 100
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [31]:
def accuracy_with_thresholding_mnist(y_true, y_pred, threshold = 0.5):

    ...

In [32]:
from torch.nn.functional import one_hot
for i in range(1):
    for idx, (x_batch, y_batch) in enumerate (train_data_loader):
        y_batch = y_batch.reshape(-1, 1)
        model.train()
        optimizer.zero_grad()
        y_pred = model(x_batch.reshape(-1, 1, 784))
        loss_value = loss(y_pred, one_hot(y_batch, 10).float())
        loss_value.backward()
        optimizer.step()
    if i % 100 == 0:
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_accuracy = 0
            for x_val, y_val in val_data_loader:
                y_pred = model(x_val.reshape(-1, 1, 784))
                y_val = y_val.reshape(-1, 1)
                y_val = one_hot(y_val, 10).float()
                val_loss = loss(y_pred, y_val)
                val_accuracy = accuracy_with_thresholding_mnist(y_val, y_pred)
            val_accuracy /= len(val_data_loader)
            val_loss /= len(val_data_loader)

            if val_accuracy > max_accuracy:
                max_accuracy = val_accuracy

            print(f"Epoch {i} - Training Loss: {loss_value.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy}")

print(loss_value)
# Using the wrong accuract_with_thresholding function

TypeError: unsupported operand type(s) for /=: 'NoneType' and 'int'

In [ ]:
x_batch.shape

In [ ]:
one_hot(y_batch).shape

torch.Size([256, 10])

In [ ]:
y_batch.shape

torch.Size([256])

In [ ]:
y_val.shape

torch.Size([40, 1, 10])

In [ ]:
y_pred.shape

torch.Size([40, 1, 10])

In [ ]:
y_pred

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [ ]:
y_val

tensor([[[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0., 

In [ ]:
y_pred = (y_pred > 0.5).int()

In [ ]:
y_pred

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1

In [ ]:
anded = torch.logical_and(y_pred, y_val)
torch.sum(anded)

tensor(40)

In [ ]:
anded.shape

torch.Size([40, 1, 10])

In [ ]:
sum_accurate = torch.sum(anded).float()

In [ ]:
sum_accurate /= len(y_pred)

In [ ]:
sum_accurate

tensor(1.)